In [ ]:
#Add word count field to discussion forum document in mongodb

from pymongo import MongoClient
from pymongo.errors import BulkWriteError

client = MongoClient()
dbname = 'crime101x2014'
db = client[dbname]

#Calculate the word count of a text variable (inteded for forum posts)
def calculate_wordcount(textcontent):
    words = ''.join(c if c.isalnum() else ' ' for c in textcontent).split()
    return len(words)

"""
Cleans up forum post data and pre processes data.
 
 Adds a Word count to the db entry
 
"""
def process_forumposts(db):
    # Find all forum posts
    cursor = db.discussion_forum.find()
    
    bulk_op = db.discussion_forum.initialize_unordered_bulk_op()

    for document in cursor:
        #print document
        mongo_id = document['_id']
        body = document['body']
        title = ""
        
        if 'title' in document: 
            title = document['title']
        #Create a vriable to hold the whole of the forum post
        forumpostcontent = body + " " + title
        #Retrieve the word count 
        wordcount = calculate_wordcount(forumpostcontent)
        #Update the db entry with a wordcount field
        bulk_op.find({'_id': mongo_id}).update({'$set': {'wordcount': wordcount}})
        
    try:
        bulk_op.execute()
    except BulkWriteError as bwe:
        print bwe.details

In [15]:
process_forumposts(db)